In [26]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as sw
import gensim
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [24]:
stopWords = set(sw.words("english"))

In [27]:
def lesk(word, sentence):
    senses =wn.synsets(word)
    best_sense= senses[0]
    max_overlap = 0
    context = set(nltk.word_tokenize(sentence))
    context = context.difference(stopWords)
    for sense in senses[1:]:
        data = set(nltk.word_tokenize(sense.definition()+" "+" ".join( sense.examples())))
        signature = data.difference(stopWords)
        overlap = len(context.intersection(signature))
        if overlap >max_overlap:
            max_overlap = overlap
            best_sense = sense
            
    return best_sense

In [28]:
lesk("bank","Yesterday I went to the bank to withdraw the money and the credit card did not work")

Synset('depository_financial_institution.n.01')